In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install tridentx --upgrade
!pip install pydicom --upgrade
!pip install python-gdcm  --upgrade
!pip install imutils
import imutils
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet
import random
from tqdm import tqdm
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import scipy

In [ ]:
dicom_images=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*/*/*.dcm')

print(len(dicom_images))

In [ ]:
flairs=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/*.dcm')
t1ws=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/*.dcm')
t1wces=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/*.dcm')
t2ws=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/*.dcm')
print( pydicom.read_file(flairs[0]))
print(pydicom.read_file(flairs[0])[0x0020, 0x0032])
print(pydicom.read_file(flairs[0])[0x0020, 0x0037])


In [ ]:
!zip -r ./train_archive.zip ./train

In [ ]:
t1ws=glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00009/T1w/*.dcm')

print(len(t1ws))
print( pydicom.read_file(t1ws[0]))
print( pydicom.read_file(t1ws[16]).pixel_array)

In [ ]:
def load_scan(path):
    """
    回傳資料夾中所有的slices
    :param path:
    :return:
    """
    g = glob.glob(path + '/*.dcm')
    slices = [pydicom.read_file(s) for s in g]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return slices

def get_pixels(slices):
    #讀出像素值並且儲存成numpy的格式
    image = np.stack([s.pixel_array for s in slices])
    check_max=np.array([im.max() for im in image])
    check_max[check_max>0]=1
    if check_max.sum()==0:
        print('check_max.sum()==0')
        return np.zeros((100,100,100)),None
        
    start=np.argmax(check_max)

    end=len(check_max)-(np.argmax(check_max[::-1]))
    if end<start:
        start,end=end,start
    
    if start>0:
        start-=1
    if end<len(check_max):
        end+=1
    print('start',start,'end',end)
    filtered_slices=slices[start:end+1]
    if len(filtered_slices)<=2:
        return np.zeros((100,100,100)),None
    
    image=image[start:end+1]
    # 將超過機器掃描範圍的部分設為 0
    # 通常intercept是 -1024, 經過計算之後空氣大約是 0
    image[image < 0] = 0
    intercept = filtered_slices[0].RescaleIntercept
    slope = filtered_slices[0].RescaleSlope
    # 轉換為Hounsfield units (HU)
    if slope != 1:
        for slice_number in range(len(filtered_slices)):

            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)

            image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16),filtered_slices

def resample(image, slices, new_spacing=[1.0, 1.0, 1.0]):
    if slices is None or (image.shape==(100,100,100) and image.sum()==0):
        return  np.zeros((100,100,100))
    #取出原本的spacing
    #[slices[0].SliceThickness] + list(slices[0].PixelSpacing) 會組成一個list，裡面是的spacing數值
    spacing = np.array([slices[0].SliceThickness] + list(slices[0].PixelSpacing), dtype=np.float64)
    
    if not hasattr(slices[0],'SliceLocation') or  not hasattr(slices[-1],'SliceLocation'):
        loc0=slices[0][('0020','0032')].value
        loc1=slices[-1][('0020','0032')].value
        idx=-1
        for i in range(3):
           if abs(loc0[i]-loc1[i])>5:
            idx=i
            
        slices[0].SliceLocation=-1*slices[0][('0020','0032')].value[idx]
        #print('loc_start',-1*slices[0][('0020','0032')].value[idx])
    
        slices[-1].SliceLocation=-1*slices[-1][('0020','0032')].value[idx]
        #print('loc_end',-1*slices[-1][('0020','0032')].value[idx])

        
    if slices[-1].SliceLocation<slices[0].SliceLocation :
        slices=slices[::-1]
    loc_start=slices[0].SliceLocation 
    loc_end=slices[-1].SliceLocation+slices[-1].SliceThickness
    print('loc_start',loc_start,'loc_end',loc_end)
    
    resize_factor = spacing / new_spacing
    new_real_shape = image.shape  * resize_factor
    
    new_shape = np.round(new_real_shape)
    
    real_resize_factor = new_shape /image.shape
    
    real_resize_factor[0]=float(builtins.min(loc_end-loc_start,256))/(image.shape[0])
    #print('real_resize_factor',real_resize_factor)
    new_spacing = spacing / real_resize_factor
    
    #計算出放大參數後，用scipy對維照片進行縮放，scipy會自動為圖片進行插補
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    image[image<0]=0
    return image


def crop_brain(image,normalized=True):
    if image is None or image.shape==(100,100,100) or image.sum()==0:
        return image
    
    flatten_image=image.max(0)
    rows=flatten_image.max(1)
    rows[rows > 0] = 1
    columns=flatten_image.max(0)
    columns[columns > 0] = 1
    y_start=np.argmax(rows)
    y_end=len(rows)-(np.argmax(rows[::-1]))
    x_start=np.argmax(columns)
    x_end=len(columns)-(np.argmax(columns[::-1]))
    if y_start>0:
        y_start-=1
    if x_start>0:
        x_start-=1
    if y_end<len(rows):
        y_end+=1
    if y_end<len(rows):
        x_end+=1
        
    crop_image=image[:,y_start:y_end+1,x_start:x_end+1]
#     if normalized:
#         mask=crop_image>0
#         hist,bins = np.histogram((crop_image[mask]).reshape(-1),bins =100)
#         base_value=bins[np.argmax(hist)]
#         crop_image=np.clip((crop_image-base_value)/base_value,-1,1)
#         crop_image[1-mask]=-1
    if normalized:
        mask=crop_image>0
        if len(crop_image[mask])>0:
            mean_value=crop_image[mask].mean()
            std_value=crop_image[mask].std()
            if std_value>0:
                crop_image=(crop_image-mean_value)/std_value
    return crop_image

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
from skimage import measure

def plot_3d(image, threshold=-300): 
    p = image.transpose(2,1,0)
    verts, faces, normals, values = measure.marching_cubes_lewiner(p, threshold)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    

    plt.show()

In [ ]:

slices = load_scan('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR')
print(len(slices))
patient_pixels,filtered_slices = get_pixels(slices)

print(patient_pixels.shape)
print(patient_pixels.min(),patient_pixels.max())
image= resample(patient_pixels, filtered_slices)

print(image.shape)
print(image.min(),image.max())
crop_image=crop_brain(image)

print(crop_image.shape)
print(crop_image.min(),crop_image.mean(),crop_image.max())


In [ ]:
print(slices[0])

In [ ]:
for k in range(len(patient_pixels)):
    slice=patient_pixels[k]
    
    mask=slice>0
    hist,bins = np.histogram(slice[mask],bins =100 )
    print(k,'peak',bins[np.argmax(hist)],'max:',slice.max(),'mean:',slice[mask].mean())
    

In [ ]:
slices = load_scan('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00095/FLAIR')
print(len(slices))
patient_pixels,filtered_slices= get_pixels(slices)

print(patient_pixels.shape)
print(patient_pixels.min(),patient_pixels.max())
image = resample(patient_pixels, filtered_slices)

print(image.shape)
print(image.min(),image.max())
crop_image=crop_brain(image,normalized=True)

print(crop_image.shape)

In [ ]:

plot_3d(image.copy(),threshold=500)

In [ ]:
#3d圖不支持ax.set_aspect('equal')，因此比例會看起來怪怪的
crop_image=crop_brain(image,normalized=False)
plot_3d(crop_image.copy(),threshold=500)

In [ ]:
slices = load_scan('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00095/T2w')
print(len(slices))
patient_pixels,filtered_slices= get_pixels(slices)
print(filtered_slices[0])

print(patient_pixels.shape)
print(patient_pixels.min(),patient_pixels.max())
image = resample(patient_pixels, filtered_slices)
image[image < 0] = 0
print(image.shape)
print(image.min(),image.max())
crop_image=crop_brain(image)

print(crop_image.shape)

In [ ]:
crop_image=crop_brain(image,normalized=False)
plot_3d(crop_image.copy(),threshold=400)

In [ ]:
crop_image=crop_brain(image,normalized=False)
plot_3d(np.flip(np.transpose(crop_image,(1,2,0)).copy(),axis=-1),threshold=400)

In [ ]:
files=glob.glob('../input/rsna-miccai-brain-tumor-baseline/train/*/*.npz')
print(len(files))

In [ ]:
import shutil
for file in files:
    try:
        new_file=file.replace('../input/rsna-miccai-brain-tumor-baseline/train','./train')
        make_dir_if_need(new_file)
        img=np.load(file)['arr_0']
        _,filename,_=split_path(new_file)
        print(filename, img.max(),img.min())
        shutil.copy(file,new_file)
        
        
    except:
        pass

    

In [ ]:
import pandas as pd
from tqdm import tqdm
df_train=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
print(df_train)

labels=[]
types=[]
images=[]
for i in tqdm(range(len(df_train['BraTS21ID']))):
    id=df_train['BraTS21ID'][i]
    img_path1='{0}/{1}_FLAIR.npz'.format(('00000'+str(id))[-5:],('00000'+str(id))[-5:])
 
    labels.append(df_train['MGMT_value'][i])
    images.append(img_path1)
    types.append(0)
    img_path2='{0}/{1}_T1w.npz'.format(('00000'+str(id))[-5:],('00000'+str(id))[-5:])

    labels.append(2+df_train['MGMT_value'][i])
    images.append(img_path2)
    types.append(1)
    img_path3='{0}/{1}_T1wCE.npz'.format(('00000'+str(id))[-5:],('00000'+str(id))[-5:])

    labels.append(4+df_train['MGMT_value'][i])
    images.append(img_path3)
    types.append(2)
    img_path4='{0}/{1}_T2w.npz'.format(('00000'+str(id))[-5:],('00000'+str(id))[-5:])

    labels.append(6+df_train['MGMT_value'][i])
    images.append(img_path4)
    types.append(3)
print(len(labels))
print(len(images))

In [ ]:
import shutil
shutil.rmtree('./train')

def prepare_image(img_path):
    start_time=time.time()
    print('start thread: {0}'.format(img_path),flush=True)
    try:
        _,filename,_=split_path(img_path)
        new_file_path=os.path.join('./train',img_path)
        make_dir_if_need(new_file_path)
        patient_id,mri_type=filename.split('_')
        if not os.path.exists(new_file_path) :
            
            if int(patient_id)>0 and int(patient_id)%10==0:
                gc.collect()
            start_time1=time.time()
            slices = load_scan(os.path.join('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train',patient_id,mri_type))
            print('Get slices: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            patient_pixels,filtered_slices= get_pixels(slices)
            print('Get pixels: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            image = resample(patient_pixels, filtered_slices)
            print('Resample: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            img=crop_brain(image)
            print('Crop image\: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)

            with open(new_file_path, 'wb') as f:
                np.savez_compressed(f, img)
            print('end thread: {0}  {1}'.format(img_path,time.time() - start_time),flush=True)
            del slices
            del filtered_slices
            del image
            del patient_pixels
            del img
            return True
        else:
            print(red_color('end thread exists: {0}  {1}'.format(img_path,time.time() - start_time)),flush=True)
            return True
    except Exception as e:
        print(red_color(e))
        img=np.zeros((100,100,100)).astype(np.float32)
        print('end thread: {0}  {1}'.format(img_path,time.time() - start_time),flush=True)
        return True

In [ ]:
for img in images:
    prepare_image(img)

from concurrent.futures import ThreadPoolExecutor



def prepare_image(img_path):
    start_time=time.time()
    print('start thread: {0}'.format(img_path),flush=True)
    try:
        _,filename,_=split_path(img_path)
        new_file_path=os.path.join('./train',img_path)
        make_dir_if_need(new_file_path)
        patient_id,mri_type=filename.split('_')
        if not os.path.exists(new_file_path) or int(patient_id)>=455:
            
            if int(patient_id)>0 and int(patient_id)%10==0:
                gc.collect()
            start_time1=time.time()
            slices = load_scan(os.path.join('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train',patient_id,mri_type))
            print('Get slices: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            patient_pixels,filtered_slices= get_pixels(slices)
            print('Get pixels: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            image = resample(patient_pixels, filtered_slices)
            print('Resample: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)
            start_time1=time.time()
            img=crop_brain(image)
            print('Crop image\: {0}  {1}'.format(img_path,time.time() - start_time1),flush=True)

            with open(new_file_path, 'wb') as f:
                np.savez_compressed(f, img)
            print('end thread: {0}  {1}'.format(img_path,time.time() - start_time),flush=True)
            del slices
            del filtered_slices
            del image
            del patient_pixels
            del img
            return True
        else:
            #print('end thread exists: {0}  {1}'.format(img_path,time.time() - start_time),flush=True)
            return True
    except Exception as e:
        print(e)
        img=np.zeros((100,100,100)).astype(np.float32)
        print('end thread: {0}  {1}'.format(img_path,time.time() - start_time),flush=True)
        return True
    
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
 
    future_to_folder = {executor.submit(prepare_image, img_path): img_path for img_path in images}
    for future in concurrent.futures.as_completed(future_to_folder):
        foler = future_to_folder[future]
        try:
            data = future.result()
            
        except Exception as exc:
            print(exc)
       
            

arr=np.load('./resample_images/00000/00000_flair.npz')['arr_0']
print(arr.shape)
print(arr.max())

from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0]/builtins.max(ims[0].max(),1), cmap="gray")

    def animate_func(i):
        im.set_array(ims[i]/builtins.max(ims[i].max(),1))
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

plt.imshow(arr[100]/arr[100].max(), cmap="gray")


create_animation(arr)